In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Capstone

Mounted at /content/drive
/content/drive/MyDrive/Capstone


In [ ]:
import pandas as pd

# 读入
tx_broker = pd.read_csv(
    "brokerage_transactions.csv",
    parse_dates=["transactionTimestamp"]
)
acct_broker = pd.read_csv("brokerage_accounts.csv")

# 合并
merged_broker = tx_broker.merge(acct_broker, on="accountId", how="left")

# 快速检查
print("【Brokerage Transactions】", tx_broker.shape)
print(tx_broker.dtypes)
print(tx_broker.isna().sum())
print("【Brokerage Accounts】", acct_broker.shape)
print(acct_broker.dtypes)
print(acct_broker.isna().sum())
print("合并后总行数:", merged_broker.shape[0])
print("缺失 accountType 数:", merged_broker["accountType"].isna().sum())
print("交易表中唯一账户数:", tx_broker["accountId"].nunique())
print("账户表中唯一账户数:", acct_broker["accountId"].nunique())


【Brokerage Transactions】 (2500, 13)
accountId                        int64
transactionId                    int64
transactionTimestamp    datetime64[ns]
description                     object
amount                         float64
units                          float64
transactionType                 object
symbol                          object
securityId                      object
securityIdType                  object
transferAction                  object
positionType                    object
status                          object
dtype: object
accountId               0
transactionId           0
transactionTimestamp    0
description             0
amount                  0
units                   0
transactionType         0
symbol                  0
securityId              0
securityIdType          0
transferAction          0
positionType            0
status                  0
dtype: int64
【Brokerage Accounts】 (100, 11)
accountId                int64
accountNumberDisplay    object

In [ ]:
# （1）保证 lastActivityDate 在 merged_broker 中是 datetime
merged_broker["lastActivityDate"] = pd.to_datetime(merged_broker["lastActivityDate"])

# （2）计算 accountAgeDays
merged_broker["accountAgeDays"] = (
    merged_broker["transactionTimestamp"] - merged_broker["lastActivityDate"]
).dt.days

# 聚合示例
feat_b = merged_broker.groupby("accountId").agg(
    tx_count         = ("transactionId",   "count"),
    tx_sum_amount    = ("amount",          "sum"),
    tx_mean_amount   = ("amount",          "mean"),
    tx_std_amount    = ("amount",          "std"),
    unique_symbols   = ("symbol",          "nunique"),
    unique_sec_types = ("securityIdType",  "nunique"),
    units_sum        = ("units",           "sum"),
    units_mean       = ("units",           "mean"),
    tx_types         = ("transactionType", "nunique"),  # 交易类型多样性
    pos_types        = ("positionType",    "nunique"),  # 持仓类型多样性
    acc_age_days     = ("accountAgeDays",  "mean")      # 平均存续时长
).reset_index()

# 把账户表的静态属性 merge 回来
feat_b = feat_b.merge(
    acct_broker[[
        "accountId","accountType","currency","margin",
        "allowedCheckWriting","status","productName"
    ]],
    on="accountId",
    how="left"
)

print(feat_b.shape)
print(feat_b.dtypes)
feat_b.head()

(100, 18)
accountId                int64
tx_count                 int64
tx_sum_amount          float64
tx_mean_amount         float64
tx_std_amount          float64
unique_symbols           int64
unique_sec_types         int64
units_sum              float64
units_mean             float64
tx_types                 int64
pos_types                int64
acc_age_days           float64
accountType             object
currency                object
margin                    bool
allowedCheckWriting       bool
status                  object
productName             object
dtype: object


,accountId,tx_count,tx_sum_amount,tx_mean_amount,tx_std_amount,unique_symbols,unique_sec_types,units_sum,units_mean,tx_types,pos_types,acc_age_days,accountType,currency,margin,allowedCheckWriting,status,productName
0,7638000000,25,-156781.10,-6271.2440,30490.320380,1,1,9801.46,392.0584,4,2,-825.64,BROKERAGE,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED
1,7638000001,25,200718.59,8028.7436,25031.794484,1,1,6992.27,279.6908,4,2,-680.60,BROKERAGE,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED
2,7638000002,25,153350.00,6134.0000,32116.422800,1,1,9478.90,379.1560,4,2,-792.48,BROKERAGE,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED
3,7638000003,25,43536.87,1741.4748,35249.801732,1,1,11405.68,456.2272,4,2,-746.80,BROKERAGE,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED
4,7638000004,25,430514.08,17220.5632,31231.433744,1,1,12452.16,498.0864,4,2,-858.68,BROKERAGE,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED


In [ ]:
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

# 1) 对应的数值特征列
num_cols = [
    "tx_count", "tx_sum_amount", "tx_mean_amount", "tx_std_amount",
    "unique_symbols", "unique_sec_types", "units_sum", "units_mean",
    "tx_types", "pos_types"
    # 不要加 "acc_age_days"
]
# 2）标准化处理
Xb = feat_b[num_cols].fillna(0)
scaler = StandardScaler()
Xb_scaled = scaler.fit_transform(Xb)

# 3) IsolationForest
iso_b = IsolationForest(contamination=0.02, random_state=42)
feat_b["iso_label"] = iso_b.fit_predict(Xb_scaled)

# 4) Local Outlier Factor
lof_b = LocalOutlierFactor(n_neighbors=20, contamination=0.02)
feat_b["lof_label"] = lof_b.fit_predict(Xb_scaled)

# 5) 交集高置信度标记
feat_b["hybrid_anomaly"] = np.where(
    (feat_b["iso_label"] == -1) & (feat_b["lof_label"] == -1),
    1, 0
)

# 6) 单笔交易 z-score 标记
def safe_z(x):
    std = x.std()
    return (x - x.mean()) / std if std > 0 else pd.Series(0, index=x.index)

merged_broker["amt_zscore"] = (
    merged_broker
    .groupby("accountId")["amount"]
    .transform(safe_z)
)
merged_broker["amt_outlier"] = merged_broker["amt_zscore"].abs() > 2.5

# 7) 标记有极端单笔交易的账户
extreme_ids_b = merged_broker.loc[merged_broker["amt_outlier"], "accountId"].unique()
feat_b["extreme_flag"] = feat_b["accountId"].isin(extreme_ids_b)

# 8) 合并所有信号，得到最终待审查
feat_b["final_anomaly"] = (
    (feat_b["iso_label"] == -1) |
    (feat_b["lof_label"] == -1) |
    (feat_b["extreme_flag"])
)

print("Brokerage 最终待审查账号数：", feat_b["final_anomaly"].sum())
display(feat_b[feat_b["final_anomaly"]])


Brokerage 最终待审查账号数： 35


,accountId,tx_count,tx_sum_amount,tx_mean_amount,tx_std_amount,unique_symbols,unique_sec_types,units_sum,units_mean,tx_types,...,currency,margin,allowedCheckWriting,status,productName,iso_label,lof_label,hybrid_anomaly,extreme_flag,final_anomaly
1,7638000001,25,200718.59,8028.7436,25031.794484,1,1,6992.27,279.6908,4,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True
2,7638000002,25,153350.00,6134.0000,32116.422800,1,1,9478.90,379.1560,4,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True
4,7638000004,25,430514.08,17220.5632,31231.433744,1,1,12452.16,498.0864,4,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,-1,0,True,True
9,7638000009,25,118118.73,4724.7492,23529.134081,1,1,8416.40,336.6560,4,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True
11,7638000011,25,147827.46,5913.0984,45381.080847,1,1,13924.61,556.9844,2,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,-1,-1,1,False,True
15,7638000015,25,-345016.67,-13800.6668,37900.648010,1,1,10044.69,401.7876,4,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True
17,7638000017,25,89379.39,3575.1756,36019.425398,1,1,10801.79,432.0716,4,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True
18,7638000018,25,124375.87,4975.0348,32413.662130,1,1,7941.52,317.6608,4,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True
22,7638000022,25,11565.87,462.6348,27577.072522,1,1,9567.49,382.6996,4,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True
23,7638000023,25,23419.45,936.7780,27682.729006,1,1,8879.02,355.1608,4,...,{'currencyCode': 'USD'},False,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True


In [ ]:
from sklearn.preprocessing import MinMaxScaler

def build_amount_sequences(merged_df):
    ids = []
    seqs = []
    for aid, df in merged_df.groupby("accountId"):
        df2 = df.sort_values("transactionTimestamp")
        seqs.append(df2["amount"].values)
        ids.append(aid)
    X = np.array(seqs)                      # (n_accounts, timesteps)
    # reshape 为 LSTM 要求的三维：[样本数, 窗口长度, 特征维度]
    X = X.reshape(X.shape[0], X.shape[1], 1)  # amount 只有 1 维
    # 归一化到 [0,1]
    scaler = MinMaxScaler()
    X_flat = X.reshape(-1, 1)
    X_flat = scaler.fit_transform(X_flat)
    X = X_flat.reshape(X.shape)
    return ids, X

In [ ]:
# 1) 构建时间序列数据
ids_b, X_b = build_amount_sequences(merged_broker)  # 见前面定义

# 2) 扁平化为二维矩阵 (n_accounts, timesteps)
X_seq_b = X_b.reshape(X_b.shape[0], X_b.shape[1])

# 3) 标准化
scaler_b = StandardScaler()
X_seq_b_scaled = scaler_b.fit_transform(X_seq_b)

In [ ]:
! pip install pyod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 15.5 MB/s eta 0:00:00


In [ ]:
# 4) 初始化并训练 AutoEncoder
from pyod.models.auto_encoder import AutoEncoder

ae_b = AutoEncoder(
    contamination=0.02,            # 2%账号异常
    preprocessing=True,            # 内部再做一次 StandardScaler
    lr=0.001,                      # 学习率
    epoch_num=50,                  # 训练轮数
    batch_size=16,                 # 批大小
    optimizer_name='adam',
    random_state=42,
    verbose=1,

    # 网络结构
    hidden_neuron_list=[64, 32, 32, 64],
    hidden_activation_name='relu',
    batch_norm=True,
    dropout_rate=0.2
)
ae_b.fit(X_seq_b_scaled)

# 5) 提取标签与分数
ae_labels_b = dict(zip(ids_b, ae_b.labels_))             # 0 正常, 1 异常
ae_scores_b = dict(zip(ids_b, ae_b.decision_scores_))    # 越大越异常

# 6) 映射回 feat_b
feat_b["ae_label"] = feat_b["accountId"].map(ae_labels_b).fillna(0).astype(int)
feat_b["ae_score"] = feat_b["accountId"].map(ae_scores_b).fillna(0)

Training: 100%|██████████| 50/50 [00:01<00:00, 31.42it/s]


In [ ]:
# 7) 更新最终待审查标记
feat_b["final_anomaly"] = (
      (feat_b["iso_label"] == -1) |
      (feat_b["lof_label"] == -1) |
      (feat_b["extreme_flag"])    |
      (feat_b["ae_label"] == 1)
)

# 8) 输出结果
print("Brokerage → 最终待审查账号数：", feat_b["final_anomaly"].sum())
display(feat_b[feat_b["final_anomaly"]])

Brokerage → 最终待审查账号数： 36


,accountId,tx_count,tx_sum_amount,tx_mean_amount,tx_std_amount,unique_symbols,unique_sec_types,units_sum,units_mean,tx_types,...,allowedCheckWriting,status,productName,iso_label,lof_label,hybrid_anomaly,extreme_flag,final_anomaly,ae_label,ae_score
1,7638000001,25,200718.59,8028.7436,25031.794484,1,1,6992.27,279.6908,4,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True,0,4.035779
2,7638000002,25,153350.00,6134.0000,32116.422800,1,1,9478.90,379.1560,4,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True,0,5.352022
4,7638000004,25,430514.08,17220.5632,31231.433744,1,1,12452.16,498.0864,4,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,-1,0,True,True,0,5.616495
9,7638000009,25,118118.73,4724.7492,23529.134081,1,1,8416.40,336.6560,4,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True,0,3.436473
11,7638000011,25,147827.46,5913.0984,45381.080847,1,1,13924.61,556.9844,2,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,-1,-1,1,False,True,1,6.706082
15,7638000015,25,-345016.67,-13800.6668,37900.648010,1,1,10044.69,401.7876,4,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True,0,6.301739
17,7638000017,25,89379.39,3575.1756,36019.425398,1,1,10801.79,432.0716,4,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True,0,5.613698
18,7638000018,25,124375.87,4975.0348,32413.662130,1,1,7941.52,317.6608,4,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True,0,4.548286
22,7638000022,25,11565.87,462.6348,27577.072522,1,1,9567.49,382.6996,4,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True,0,4.441295
23,7638000023,25,23419.45,936.7780,27682.729006,1,1,8879.02,355.1608,4,...,False,OPEN,SYNTHETIC COMPANY SELF-DIRECTED,1,1,0,True,True,0,4.259929


## DEMO

In [ ]:
# === Brokerage anomaly review mini-demo (ipywidgets) ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as w
from IPython.display import display, clear_output

# ---------- Config (与主流程对齐) ----------
IF_CONT = 0.02
LOF_CONT = 0.02
AE_CONT  = 0.02
RANDOM_SEED = 42

plt.rcParams["figure.figsize"] = (8, 2.8)

# ---------- Helpers ----------
def _safe_col(df, candidates, default=None):
    for c in candidates:
        if c in df.columns:
            return c
    return default

def _ensure_final_anomaly(feat_df):
    """若没有 final_anomaly，用现有标记合成一列。"""
    if "final_anomaly" in feat_df.columns:
        return feat_df
    flags = []
    if "iso_label" in feat_df.columns: flags.append(feat_df["iso_label"] == -1)
    if "lof_label" in feat_df.columns: flags.append(feat_df["lof_label"] == -1)
    if "extreme_flag" in feat_df.columns: flags.append(feat_df["extreme_flag"].astype(bool))
    if "ae_label"   in feat_df.columns: flags.append(feat_df["ae_label"]   == 1)
    if flags:
        feat_df = feat_df.copy()
        feat_df["final_anomaly"] = np.logical_or.reduce(flags)
    else:
        raise ValueError("No anomaly flags found on feat_b. Please run detection steps first.")
    return feat_df

def _merge_uploaded_brokerage(tx_df, ac_df):
    """上传 CSV 的合并：解析时间戳+按 accountId 左联。"""
    ts_col = _safe_col(tx_df, ["transactionTimestamp", "timestamp", "time"])
    if ts_col is None:
        raise ValueError("Cannot find a timestamp column (e.g., 'transactionTimestamp') in transactions CSV.")
    if not np.issubdtype(tx_df[ts_col].dtype, np.datetime64):
        tx_df[ts_col] = pd.to_datetime(tx_df[ts_col], errors="coerce")

    if "accountId" not in tx_df.columns or "accountId" not in ac_df.columns:
        raise ValueError("Both CSVs must include 'accountId'.")

    merged_df = tx_df.merge(ac_df, on="accountId", how="left", suffixes=("_tx", "_acct"))
    return merged_df, ts_col

def _equalize_sequences(seqs, take="min"):
    """把不同长度的金额序列对齐到同一个长度，默认取所有账户的最短长度的末尾部分。"""
    lengths = [len(s) for s in seqs]
    if len(set(lengths)) == 1:
        return np.asarray(seqs), lengths[0]
    L = min(lengths) if take == "min" else max(lengths)
    # 取每个账户最近的 L 条
    seqs_trim = [np.asarray(s[-L:]) if len(s) >= L else np.pad(np.asarray(s), (L-len(s),0), 'constant', constant_values=0.0)
                 for s in seqs]
    return np.stack(seqs_trim, axis=0), L

def _show_account_detail(merged_df, account_id):
    df = merged_df[merged_df["accountId"] == account_id].copy()
    if df.empty:
        print("No transactions for this account.")
        return
    ts_col   = _safe_col(df, ["transactionTimestamp", "timestamp", "time"])
    type_col = _safe_col(df, ["transactionType", "type"])
    desc_col = _safe_col(df, ["description_x", "description_tx", "description"])
    sym_col  = _safe_col(df, ["symbol"])
    units_col = _safe_col(df, ["units"])

    cols = ["accountId"]
    if ts_col: cols.append(ts_col)
    if "transactionId" in df.columns: cols.append("transactionId")
    cols += ["amount"]
    if units_col: cols.append(units_col)
    if type_col: cols.append(type_col)
    if sym_col:  cols.append(sym_col)
    if desc_col: cols.append(desc_col)

    display(df.sort_values("amount", ascending=False)[cols].head(20).reset_index(drop=True))

    if ts_col:
        df2 = df.sort_values(ts_col)
        plt.plot(df2[ts_col], df2["amount"], marker="o")
        plt.title(f"Account {account_id} — Amount over Time")
        plt.xlabel("Time"); plt.ylabel("Amount"); plt.xticks(rotation=45)
        plt.tight_layout(); plt.show()

# ---------- 完整检测（与主流程一致） ----------
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

def full_detect_on_uploaded_brokerage(merged_df, use_ae=True,
                                      if_cont=IF_CONT, lof_cont=LOF_CONT, ae_cont=AE_CONT,
                                      random_state=RANDOM_SEED):
    """
    Brokerage 完整检测：特征聚合 → StandardScaler → IF/LOF → 单笔Z分 → (可选) AE → final_anomaly
    """
    # 1) 账户级聚合特征（与 feat_b 对齐）
    agg = dict(
        tx_count=("transactionId", "count") if "transactionId" in merged_df.columns else ("amount","count"),
        tx_sum_amount=("amount","sum"),
        tx_mean_amount=("amount","mean"),
        tx_std_amount=("amount","std"),
        unique_symbols=("symbol","nunique") if "symbol" in merged_df.columns else ("accountId","nunique"),
        unique_sec_types=("securityIdType","nunique") if "securityIdType" in merged_df.columns else ("accountId","nunique"),
        units_sum=("units","sum") if "units" in merged_df.columns else ("amount","sum"),
        units_mean=("units","mean") if "units" in merged_df.columns else ("amount","mean"),
        tx_types=("transactionType","nunique") if "transactionType" in merged_df.columns else ("accountId","nunique"),
        pos_types=("positionType","nunique") if "positionType" in merged_df.columns else ("accountId","nunique"),
    )
    feat_up = merged_df.groupby("accountId").agg(**agg).reset_index()

    # 2) 单笔金额 z-score → 极端交易账号（Brokerage 金额可负，Z分OK）
    def safe_z(x):
        s = x.std()
        return (x - x.mean())/s if (s is not None and s>0) else pd.Series(0, index=x.index)

    tmp = merged_df.copy()
    tmp["amt_z"] = tmp.groupby("accountId")["amount"].transform(safe_z)
    tmp["amt_outlier"] = tmp["amt_z"].abs() > 2.5
    extreme_ids = tmp.loc[tmp["amt_outlier"], "accountId"].unique().tolist()
    feat_up["extreme_flag"] = feat_up["accountId"].isin(extreme_ids)

    # 3) IF/LOF（Brokerage 用 StandardScaler，避免 log1p 的负数问题）
    num_cols_b = ["tx_count","tx_sum_amount","tx_mean_amount","tx_std_amount",
                  "unique_symbols","unique_sec_types","units_sum","units_mean",
                  "tx_types","pos_types"]
    X = feat_up[num_cols_b].fillna(0).values
    X_scaled = StandardScaler().fit_transform(X)

    iso = IsolationForest(contamination=if_cont, random_state=random_state, n_estimators=200)
    feat_up["iso_label"] = iso.fit_predict(X_scaled)
    feat_up["iso_score"] = iso.decision_function(X_scaled)

    lof = LocalOutlierFactor(n_neighbors=20, contamination=lof_cont)
    feat_up["lof_label"] = lof.fit_predict(X_scaled)
    feat_up["lof_score"]  = -lof.negative_outlier_factor_

    # 4) AutoEncoder（PyOD，按主流程）
    if use_ae:
        # 构建金额序列，并对齐长度
        ts_col = _safe_col(merged_df, ["transactionTimestamp", "timestamp", "time"])
        df_sorted = merged_df.sort_values(["accountId", ts_col]) if ts_col else merged_df.sort_values(["accountId"])
        ids, seqs = [], []
        for aid, df in df_sorted.groupby("accountId"):
            ids.append(aid)
            seqs.append(df["amount"].values)
        X_seq, L = _equalize_sequences(seqs, take="min")  # shape: (n_accounts, L)
        X_seq_scaled = StandardScaler().fit_transform(X_seq)

        from pyod.models.auto_encoder import AutoEncoder
        ae = AutoEncoder(
            contamination=ae_cont,
            preprocessing=True,
            lr=0.001,
            epoch_num=50,
            batch_size=16,
            optimizer_name='adam',
            random_state=random_state,
            verbose=0,
            hidden_neuron_list=[64,32,32,64],
            hidden_activation_name='relu',
            batch_norm=True,
            dropout_rate=0.2
        )
        ae.fit(X_seq_scaled)
        ae_labels = dict(zip(ids, ae.labels_))
        ae_scores = dict(zip(ids, ae.decision_scores_))
        feat_up["ae_label"] = feat_up["accountId"].map(ae_labels).fillna(0).astype(int)
        feat_up["ae_score"] = feat_up["accountId"].map(ae_scores).fillna(0)
    else:
        feat_up["ae_label"] = 0
        feat_up["ae_score"] = 0.0

    # 5) 最终标记（与主流程一致的 OR 规则；如需严格可改成 2/3 多数决）
    feat_up["final_anomaly"] = (
        (feat_up["iso_label"] == -1) |
        (feat_up["lof_label"] == -1) |
        (feat_up["extreme_flag"])    |
        (feat_up["ae_label"] == 1)
    )
    return feat_up

# ---------- UI ----------
st_source = w.ToggleButtons(
    options=[("Use in-memory (merged_broker/feat_b)", "mem"),
             ("Upload CSVs (transactions + accounts)", "upload")],
    description="Source:",
)
use_ae_ck = w.Checkbox(value=True, description="Use AutoEncoder (slower)")
u_tx = w.FileUpload(accept=".csv", multiple=False, description="Upload brokerage transactions CSV")
u_ac = w.FileUpload(accept=".csv", multiple=False, description="Upload brokerage accounts CSV")
btn_run = w.Button(description="Run brokerage review", button_style="primary")
out = w.Output()

def on_run(_):
    with out:
        clear_output()
        try:
            if st_source.value == "mem":
                # —— 用内存里的 merged_broker / feat_b ——
                if "merged_broker" not in globals() or "feat_b" not in globals():
                    print("Could not find 'merged_broker' and 'feat_b' in memory. "
                          "Switch to 'Upload CSVs' or run the earlier pipeline cells.")
                    return
                merged_df = merged_broker.copy()
                feat_df   = feat_b.copy()
                # 确保 final_anomaly 存在
                try:
                    feat_df = _ensure_final_anomaly(feat_df)
                except Exception as e:
                    print(f"{e}\nShowing aggregates only.")
            else:
                # —— 上传 CSVs 跑完整检测（与主流程对齐） ——
                if (len(u_tx.value) == 0) or (len(u_ac.value) == 0):
                    print("Please upload both transactions and accounts CSV files.")
                    return
                tx_bytes = next(iter(u_tx.value.values()))["content"]
                ac_bytes = next(iter(u_ac.value.values()))["content"]
                tx_df = pd.read_csv(pd.io.common.BytesIO(tx_bytes))
                ac_df = pd.read_csv(pd.io.common.BytesIO(ac_bytes))

                merged_df, _ = _merge_uploaded_brokerage(tx_df, ac_df)
                feat_df = full_detect_on_uploaded_brokerage(
                    merged_df,
                    use_ae=use_ae_ck.value,
                    if_cont=IF_CONT, lof_cont=LOF_CONT, ae_cont=AE_CONT,
                    random_state=RANDOM_SEED
                )
                print("Full detection finished on uploaded brokerage data.")

            # —— 展示异常账户列表 ——
            if "final_anomaly" in feat_df.columns and feat_df["final_anomaly"].any():
                anom = feat_df[feat_df["final_anomaly"]].copy()
                print(f"Accounts flagged for review: {len(anom)}")
                keep_cols = [c for c in [
                    "iso_score","lof_score","ae_score","extreme_flag",
                    "tx_sum_amount","tx_mean_amount","tx_std_amount","tx_count",
                    "unique_symbols","unique_sec_types","units_sum","units_mean","tx_types","pos_types"
                ] if c in anom.columns]
                display(anom[["accountId"] + keep_cols].reset_index(drop=True))

                # 选择账户看详情
                acc_dd = w.Dropdown(options=anom["accountId"].tolist(), description="Account:")
                btn_show = w.Button(description="Show detail")
                box2 = w.HBox([acc_dd, btn_show])
                display(box2)

                def _on_show(__):
                    with out:
                        clear_output(wait=True)
                        print(f"Accounts flagged for review: {len(anom)}")
                        display(anom[["accountId"] + keep_cols].reset_index(drop=True))
                        display(box2)
                        _show_account_detail(merged_df, acc_dd.value)
                btn_show.on_click(_on_show)
            else:
                print("No accounts currently flagged (or no flags available).")
                # 仍允许浏览任意账户
                all_ids = merged_df["accountId"].unique().tolist()
                if not all_ids:
                    return
                acc_dd = w.Dropdown(options=all_ids[:50], description="Account:")
                btn_show = w.Button(description="Show detail")
                display(w.HBox([acc_dd, btn_show]))
                def _on_show_any(__):
                    with out:
                        clear_output(wait=True)
                        _show_account_detail(merged_df, acc_dd.value)
                btn_show.on_click(_on_show_any)
        except Exception as e:
            print("Error:", e)

btn_run.on_click(on_run)
display(w.VBox([w.HBox([st_source, use_ae_ck]), w.HBox([u_tx, u_ac]), btn_run, out]))
